In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split,GridSearchCV
import pickle
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import accuracy_score, roc_curve, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import json
# from sklearn.ensemble import CatBoostClassifier

In [ ]:
import pandas as pd
df = pd.read_csv('1_2may_testing_labeled.csv',dtype={"user_pseudo_id":str})
df.shape

In [4]:
df.head()

,ProspectId,user_pseudo_id,visitStartTime,event_date,day_of_week_past,timeOnSite_past,device_category_past,mobile_brand_name_past,operating_system_past,os_version_past,...,city_past,ft_campaign_past,ft_source_past,ft_medium_past,utm_term_placement_past,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion,label
0,003e3eb8-a499-4e94-a0a4-a1f90d65c7aa,834139000.2,1683007316,20230502,3,0.000000,mobile,Infinix,Android,Android 11,...,Patna,(direct),(direct),(none),NaN,ringing (pan card not available),Not Contactable,Not Contactable,Ringing,0
1,008a83af-1a2a-4528-87d3-a5af33013ad6,353319200.2,1682899533,20230501,2,1.474086,mobile,Huawei,Android,Android 6.0.0,...,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,NaN,NaN,Not Interested,NaN,Others,0
2,008a83af-1a2a-4528-87d3-a5af33013ad6,353319200.2,1682899533,20230501,2,0.188153,mobile,Huawei,Android,Android 6.0.0,...,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,NaN,NaN,Not Interested,NaN,Others,0
3,008a83af-1a2a-4528-87d3-a5af33013ad6,353319200.2,1682899533,20230501,2,1.897437,mobile,Huawei,Android,Android 6.0.0,...,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,NaN,NaN,Not Interested,NaN,Others,0
4,008a83af-1a2a-4528-87d3-a5af33013ad6,353319200.2,1682916115,20230501,2,21.324201,mobile,Huawei,Android,Android 6.0.0,...,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,NaN,NaN,Not Interested,NaN,Others,0


In [5]:
df['ft_campaign_past'].value_counts()

(direct)                                                                               2297
Display_Prospecting_Demat_Tier1                                                        1486
Leads_Performance_Demat_Tier_2                                                         1442
Leads_Performance_Max_FandO_Tier_1                                                     1230
Display_Prospecting_Fando_Tier1                                                        1014
Display_Prospecting_Advisory_Tier1                                                      990
Display_Prospecting_Intraday_Tier1                                                      831
link                                                                                    512
Leads_Performance_Max_Advisory_Tier_1                                                   392
Display_Prospecting_MTF_Tier1                                                           304
Display_Manual_Placements_Demat_Tier1                                           

In [6]:
df.isnull().sum()

ProspectId                            0
user_pseudo_id                        0
visitStartTime                        0
event_date                            0
day_of_week_past                      0
timeOnSite_past                       0
device_category_past                  0
mobile_brand_name_past              236
operating_system_past                 0
os_version_past                       0
browser_past                          0
region_past                          34
city_past                           149
ft_campaign_past                     48
ft_source_past                        0
ft_medium_past                        0
utm_term_placement_past           10414
mx_Last_Activity_Notes             3716
ProspectActivityName_Max             25
mx_Last_Activity_done_by_Agent    11681
mx_Sub_Dispostion                  1738
label                                 0
dtype: int64

In [7]:
int_cols = list(df.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [8]:
cat_cols = list(df.select_dtypes(include=['O']).columns)
float_cols = list(df.select_dtypes(include=[float]).columns)

In [9]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [10]:
def analysis_count(column_name,number):
    return df.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [11]:
##data -> dictionary 
# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]
data['device_category_past'] =  analysis_column('device_category_past',4)
data['device_category_past'] = [i for i in data['device_category_past']  if i not in unwanted] 
data['mobile_brand_name_past'] =  analysis_column('mobile_brand_name_past',10)
data['mobile_brand_name_past'] = [i for i in data['mobile_brand_name_past']  if i not in unwanted] 
data['operating_system_past'] =  analysis_column('operating_system_past',5)
data['operating_system_past'] = [i for i in data['operating_system_past']  if i not in unwanted] 
data['os_version_past'] =  analysis_column('os_version_past',10)
data['os_version_past'] = [i for i in data['os_version_past']  if i not in unwanted] 
data['browser_past'] =  analysis_column('browser_past',8)
data['browser_past'] = [i for i in data['browser_past']  if i not in unwanted]
data['city_past'] =  analysis_column('city_past',160)
data['city_past'] = [i for i in data['city_past']  if i not in unwanted]
data['region_past'] =  analysis_column('region_past',50)
data['region_past'] = [i for i in data['region_past']  if i not in unwanted] 
data['ft_campaign_past'] =  analysis_column('ft_campaign_past',70)
data['ft_campaign_past'] = [i for i in data['ft_campaign_past']  if i not in unwanted]
data['ft_medium_past'] =  analysis_column('ft_medium_past',5)
data['ft_medium_past'] = [i for i in data['ft_medium_past']  if i not in unwanted] 
data['ft_source_past'] =  analysis_column('ft_source_past',10)
data['ft_source_past'] = [i for i in data['ft_source_past']  if i not in unwanted] 
data['utm_term_placement_past'] =  analysis_column('utm_term_placement_past',200)
data['utm_term_placement_past'] = [i for i in data['utm_term_placement_past']  if i not in unwanted]
# data['placement_past'] =  analysis_column('placement_past',150)
# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [12]:
# df['placement'].loc[df['placement'].apply(lambda x: (x not in data['placement']) )] = '(Others)'
df['device_category_past'].loc[df['device_category_past'].apply(lambda x: (x not in data['device_category_past']) )] = '(Others)'
df['mobile_brand_name_past'].loc[df['mobile_brand_name_past'].apply(lambda x: (x not in data['mobile_brand_name_past']) )] = '(Others)'
df['operating_system_past'].loc[df['operating_system_past'].apply(lambda x: (x not in data['operating_system_past']) )] = '(Others)'
df['os_version_past'].loc[df['os_version_past'].apply(lambda x: (x not in data['os_version_past']) )] = '(Others)'
df['browser_past'].loc[df['browser_past'].apply(lambda x: (x not in data['browser_past']) )] = '(Others)'
df['city_past'].loc[df['city_past'].apply(lambda x: (x not in data['city_past']) )] = '(Others)'
df['region_past'].loc[df['region_past'].apply(lambda x: (x not in data['region_past']) )] = '(Others)'
df['ft_campaign_past'].loc[df['ft_campaign_past'].apply(lambda x: (x not in data['ft_campaign_past']) )] = '(Others)'
df['ft_medium_past'].loc[df['ft_medium_past'].apply(lambda x: (x not in data['ft_medium_past']) )] = '(Others)'
df['ft_source_past'].loc[df['ft_source_past'].apply(lambda x: (x not in data['ft_source_past']) )] = '(Others)'
df['utm_term_placement_past'].loc[df['utm_term_placement_past'].apply(lambda x: (x not in data['utm_term_placement_past']) )] = '(Others)'
# df['placement_past'].loc[df['placement_past'].apply(lambda x: (x not in data['placement_past']) )] = '(Others)'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
df.head()

,ProspectId,user_pseudo_id,visitStartTime,event_date,day_of_week_past,timeOnSite_past,device_category_past,mobile_brand_name_past,operating_system_past,os_version_past,...,city_past,ft_campaign_past,ft_source_past,ft_medium_past,utm_term_placement_past,mx_Last_Activity_Notes,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion,label
0,003e3eb8-a499-4e94-a0a4-a1f90d65c7aa,834139000.2,1683007316,20230502,3,0.000000,mobile,Infinix,Android,Android 11,...,Patna,(Others),(Others),(Others),(Others),ringing (pan card not available),Not Contactable,Not Contactable,Ringing,0
1,008a83af-1a2a-4528-87d3-a5af33013ad6,353319200.2,1682899533,20230501,2,1.474086,mobile,(Others),Android,(Others),...,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,(Others),NaN,Not Interested,NaN,Others,0
2,008a83af-1a2a-4528-87d3-a5af33013ad6,353319200.2,1682899533,20230501,2,0.188153,mobile,(Others),Android,(Others),...,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,(Others),NaN,Not Interested,NaN,Others,0
3,008a83af-1a2a-4528-87d3-a5af33013ad6,353319200.2,1682899533,20230501,2,1.897437,mobile,(Others),Android,(Others),...,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,(Others),NaN,Not Interested,NaN,Others,0
4,008a83af-1a2a-4528-87d3-a5af33013ad6,353319200.2,1682916115,20230501,2,21.324201,mobile,(Others),Android,(Others),...,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,(Others),NaN,Not Interested,NaN,Others,0


In [14]:
### replace the string int and float value
for i in cat_cols:
    df[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df[i].fillna(-99999999.000, inplace = True) 

In [17]:
drop_cols = ['ProspectId', 'user_pseudo_id', 'visitStartTime', 'event_date',
       'device_category_past','mx_Last_Activity_Notes']
df = df.drop(drop_cols, axis=1)

In [18]:
df.head()

,day_of_week_past,timeOnSite_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_source_past,ft_medium_past,utm_term_placement_past,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion,label
0,3,0.000000,Infinix,Android,Android 11,Chrome,Bihar,Patna,(Others),(Others),(Others),(Others),Not Contactable,Not Contactable,Ringing,0
1,2,1.474086,(Others),Android,(Others),Chrome,Uttar Pradesh,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,(Others),Not Interested,Value_Not_Found,Others,0
2,2,0.188153,(Others),Android,(Others),Chrome,Uttar Pradesh,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,(Others),Not Interested,Value_Not_Found,Others,0
3,2,1.897437,(Others),Android,(Others),Chrome,Uttar Pradesh,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,(Others),Not Interested,Value_Not_Found,Others,0
4,2,21.324201,(Others),Android,(Others),Chrome,Uttar Pradesh,Lucknow,Leads_Performance_Demat_Tier_2,google,cpc,(Others),Not Interested,Value_Not_Found,Others,0


In [19]:
df['label'].value_counts()

0    11987
1      104
Name: label, dtype: int64

In [20]:
df.columns

Index(['day_of_week_past', 'timeOnSite_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_source_past',
       'ft_medium_past', 'utm_term_placement_past', 'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion', 'label'],
      dtype='object')

In [21]:
cat_cols = list(df.select_dtypes(include=['O']).columns)
cat_cols

['mobile_brand_name_past',
 'operating_system_past',
 'os_version_past',
 'browser_past',
 'region_past',
 'city_past',
 'ft_campaign_past',
 'ft_source_past',
 'ft_medium_past',
 'utm_term_placement_past',
 'ProspectActivityName_Max',
 'mx_Last_Activity_done_by_Agent',
 'mx_Sub_Dispostion']

In [22]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df[i] = le.fit_transform(df[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [23]:
X1= df.drop(['label'], axis=1)
y1 = df.label

In [24]:
y1=np.floor(pd.to_numeric(y1, errors='coerce')).astype('Int64')
import numpy as np
y1 = y1.fillna(0).astype(np.int64, errors='ignore')
X1 = X1.fillna(0).astype(np.int64, errors='ignore')

In [25]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X1, y1 = oversample.fit_resample(X1, y1)

In [26]:
X_train, X_test, y_train, y_test=train_test_split(X1, y1, test_size=.25, stratify=y1)

In [27]:
from sklearn.preprocessing import StandardScaler

ss_train = MinMaxScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = MinMaxScaler()
X_test = ss_test.fit_transform(X_test)

In [28]:
X_train

array([[0.        , 0.        , 0.7       , ..., 0.75      , 1.        ,
        0.48958333],
       [1.        , 0.        , 0.3       , ..., 0.75      , 1.        ,
        0.54166667],
       [1.        , 0.00696864, 0.6       , ..., 0.        , 1.        ,
        0.08333333],
       ...,
       [1.        , 0.09059233, 0.5       , ..., 0.625     , 1.        ,
        0.0625    ],
       [0.        , 0.        , 0.1       , ..., 0.5625    , 1.        ,
        0.16666667],
       [1.        , 0.        , 0.6       , ..., 0.75      , 1.        ,
        0.41666667]])

In [29]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [30]:
clf1 = RandomForestClassifier(bootstrap=False, max_depth=70, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=1200)
clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=70, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=1200)

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score

y_pred = clf1.predict(X_train)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y_train, y_pred)
print("accuracy : ", accuracy_score(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))
print("roc auc : ", roc_auc_score(y_train, y_pred))
print("recall : ", recall_score(y_train, y_pred))
print("precision : ", precision_score(y_train, y_pred))
print("f1 score : ",f1 )

print ("----------------------")


y_pred = clf1.predict(X_test)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y_test, y_pred)
print("accuracy : ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("roc auc : ", roc_auc_score(y_test, y_pred))
print("recall : ", recall_score(y_test, y_pred))
print("precision : ", precision_score(y_test, y_pred))
print("f1 score : ",f1)

accuracy :  0.9980533926585095
[[8956   34]
 [   1 8989]]
roc auc :  0.9980533926585093
recall :  0.999888765294772
precision :  0.9962318519339466
f1 score :  0.9980569588630434
----------------------
accuracy :  0.9971638304971638
[[2982   15]
 [   2 2995]]
roc auc :  0.9971638304971638
recall :  0.9993326659993327
precision :  0.9950166112956811
f1 score :  0.9971699683702347


In [32]:
clf1


RandomForestClassifier(bootstrap=False, max_depth=70, max_features='sqrt',
                       min_samples_leaf=2, min_samples_split=10,
                       n_estimators=1200)

In [56]:
filename = 'model/RF_Dhawal.pkl'

pickle.dump(clf1, open(filename, 'wb'))

In [38]:
#Predicting the test set result  
y_pred= clf1.predict(X_test) 
y_pred1= clf1.predict_proba(X_test) 

In [39]:
y_pred=y_test

In [40]:
y_pred1=y_pred1[:,1]

In [41]:
y_pred=list(y_pred)

In [42]:
y_pred1
for i in range(y_pred1.size):
    if y_pred1[i]>0.6250:
        y_pred1[i]=1
    elif y_pred1[i]<0.2750:
        y_pred1[i]=0
    else:
        y_pred1[i]=-1

In [43]:
y_pred1

array([0., 0., 0., ..., 0., 1., 0.])

In [44]:
a=[]
b=[]
for i in range(y_pred1.size):
    if y_pred1[i]==1:
        a.append(y_pred1[i])
        b.append(y_pred[i])
    elif y_pred1[i]==0:
        a.append(y_pred1[i])
        b.append(y_pred[i])
        

In [45]:
len(a)

42645

In [46]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(a, b)

TN, FP, FN, TP = confusion_matrix(a, b).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

True Positive(TP)  =  20673
False Positive(FP) =  396
True Negative(TN)  =  20890
False Negative(FN) =  686


In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score



print ("----------------------")


print("accuracy : ", accuracy_score(a, b))
print(confusion_matrix(a, b))
print("roc auc : ", roc_auc_score(a, b))
print("recall : ", recall_score(a, b))
print("precision : ", precision_score(a, b))


----------------------
accuracy :  0.9746277406495486
[[20890   396]
 [  686 20673]]
roc auc :  0.9746393071836105
recall :  0.9678823914977293
precision :  0.9812046134130713


In [48]:
f1_score(a,b)

0.9744979730366738

# Production 

In [44]:
client = bigquery.Client()

print("Client creating using default project: {}".format(client.project))
query = """
 select * from `jm-ebg.EDA_MODEL.lable_data_with_extra_features`
 """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    # location="US",
)  # API request - starts the query
df3 = query_job.to_dataframe()
df3.shape
# df.to_csv('train_data.csv')

Client creating using default project: jm-ebg


(102571, 28)

In [45]:
int_cols = list(df3.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [46]:
cat_cols = list(df3.select_dtypes(include=['O']).columns)
float_cols = list(df3.select_dtypes(include=[float]).columns)

In [47]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [48]:
def analysis_count(column_name,number):
    return df3.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df3.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [49]:
##data -> dictionary 
# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]
data['device_category_past'] =  analysis_column('device_category_past',4)
data['device_category_past'] = [i for i in data['device_category_past']  if i not in unwanted] 
data['mobile_brand_name_past'] =  analysis_column('mobile_brand_name_past',10)
data['mobile_brand_name_past'] = [i for i in data['mobile_brand_name_past']  if i not in unwanted] 
data['operating_system_past'] =  analysis_column('operating_system_past',5)
data['operating_system_past'] = [i for i in data['operating_system_past']  if i not in unwanted] 
data['os_version_past'] =  analysis_column('os_version_past',10)
data['os_version_past'] = [i for i in data['os_version_past']  if i not in unwanted] 
data['browser_past'] =  analysis_column('browser_past',8)
data['browser_past'] = [i for i in data['browser_past']  if i not in unwanted]
data['city_past'] =  analysis_column('city_past',160)
data['city_past'] = [i for i in data['city_past']  if i not in unwanted]
data['region_past'] =  analysis_column('region_past',50)
data['region_past'] = [i for i in data['region_past']  if i not in unwanted] 
data['ft_campaign_past'] =  analysis_column('ft_campaign_past',70)
data['ft_campaign_past'] = [i for i in data['ft_campaign_past']  if i not in unwanted]
data['ft_medium_past'] =  analysis_column('ft_medium_past',5)
data['ft_medium_past'] = [i for i in data['ft_medium_past']  if i not in unwanted] 
data['ft_source_past'] =  analysis_column('ft_source_past',10)
data['ft_source_past'] = [i for i in data['ft_source_past']  if i not in unwanted] 
data['utm_term_placement_past'] =  analysis_column('utm_term_placement_past',200)
data['utm_term_placement_past'] = [i for i in data['utm_term_placement_past']  if i not in unwanted]
# data['placement_past'] =  analysis_column('placement_past',150)
# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [50]:
# df['placement'].loc[df['placement'].apply(lambda x: (x not in data['placement']) )] = '(Others)'
df3['device_category_past'].loc[df3['device_category_past'].apply(lambda x: (x not in data['device_category_past']) )] = '(Others)'
df3['mobile_brand_name_past'].loc[df3['mobile_brand_name_past'].apply(lambda x: (x not in data['mobile_brand_name_past']) )] = '(Others)'
df3['operating_system_past'].loc[df3['operating_system_past'].apply(lambda x: (x not in data['operating_system_past']) )] = '(Others)'
df3['os_version_past'].loc[df3['os_version_past'].apply(lambda x: (x not in data['os_version_past']) )] = '(Others)'
df3['browser_past'].loc[df3['browser_past'].apply(lambda x: (x not in data['browser_past']) )] = '(Others)'
df3['city_past'].loc[df3['city_past'].apply(lambda x: (x not in data['city_past']) )] = '(Others)'
df3['region_past'].loc[df3['region_past'].apply(lambda x: (x not in data['region_past']) )] = '(Others)'
df3['ft_campaign_past'].loc[df3['ft_campaign_past'].apply(lambda x: (x not in data['ft_campaign_past']) )] = '(Others)'
df3['ft_medium_past'].loc[df3['ft_medium_past'].apply(lambda x: (x not in data['ft_medium_past']) )] = '(Others)'
df3['ft_source_past'].loc[df3['ft_source_past'].apply(lambda x: (x not in data['ft_source_past']) )] = '(Others)'
df3['utm_term_placement_past'].loc[df3['utm_term_placement_past'].apply(lambda x: (x not in data['utm_term_placement_past']) )] = '(Others)'
# df['placement_past'].loc[df['placement_past'].apply(lambda x: (x not in data['placement_past']) )] = '(Others)'

In [51]:
### replace the string int and float value
for i in cat_cols:
    df3[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df3[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df3[i].fillna(-99999999.000, inplace = True) 

In [52]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df3[i] = le.fit_transform(df3[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [53]:
drop_cols = ['ProspectId', 'user_pseudo_id', 'visitStartTime', 'event_date','user_psudo_id_conv', 'ProspectId_conv', 'visitStartTime_conv',
       'date_conv', 'day_diff','device_category_past','ProspectID_LeadCreation','mx_Last_Activity_Notes']
df3 = df3.drop(drop_cols, axis=1)

In [54]:
df3.columns

Index(['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past', 'label',
       'ProspectActivityName_Max', 'mx_Last_Activity_done_by_Agent',
       'mx_Sub_Dispostion'],
      dtype='object')

In [55]:
df.columns

Index(['day_of_week_past', 'timeOnSite_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_source_past',
       'ft_medium_past', 'utm_term_placement_past', 'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion', 'label'],
      dtype='object')

In [64]:
df.head()

,timeOnSite_past,day_of_week_past,mobile_brand_name_past,operating_system_past,os_version_past,browser_past,region_past,city_past,ft_campaign_past,ft_medium_past,ft_source_past,utm_term_placement_past,label,ProspectActivityName_Max,mx_Last_Activity_done_by_Agent,mx_Sub_Dispostion
0,0.682301,5,0,4,0,5,46,3,48,1,6,71,0,12,8,62
1,35.725837,1,4,1,5,2,26,102,0,0,0,0,0,15,8,20
2,0.462754,4,6,1,3,2,42,32,39,1,6,142,0,15,8,62
3,24.002012,6,9,1,3,2,43,61,65,1,6,0,0,15,2,113
4,4.844952,6,9,1,6,2,26,102,0,0,0,0,0,12,8,20


In [56]:
X = df3.drop(['label'], axis=1)
y = df3.label

In [58]:
X=X.loc[:,['day_of_week_past', 'timeOnSite_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_source_past',
       'ft_medium_past', 'utm_term_placement_past', 'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion',]]

In [59]:
X.columns

Index(['day_of_week_past', 'timeOnSite_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_source_past',
       'ft_medium_past', 'utm_term_placement_past', 'ProspectActivityName_Max',
       'mx_Last_Activity_done_by_Agent', 'mx_Sub_Dispostion'],
      dtype='object')

In [69]:
df.columns

Index(['timeOnSite_past', 'day_of_week_past', 'mobile_brand_name_past',
       'operating_system_past', 'os_version_past', 'browser_past',
       'region_past', 'city_past', 'ft_campaign_past', 'ft_medium_past',
       'ft_source_past', 'utm_term_placement_past', 'label',
       'ProspectActivityName_Max', 'mx_Last_Activity_done_by_Agent',
       'mx_Sub_Dispostion'],
      dtype='object')

In [60]:
y=np.floor(pd.to_numeric(y, errors='coerce')).astype('Int64')
import numpy as np
y = y.fillna(0).astype(np.int64, errors='ignore')
X = X.fillna(0).astype(np.int64, errors='ignore')

In [61]:
ss_train = MinMaxScaler()
X= ss_train.fit_transform(X)

In [101]:
filename = 'model/RF_Dhawal.pkl'
model = pickle.load(open(filename, 'rb'))

In [62]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score

y_pred = clf1.predict(X)
#y_pred = [1 if val >= 0.50 else 0 for val in y_pred_prob[:,1]]
f1 = f1_score(y, y_pred)
print("accuracy : ", accuracy_score(y, y_pred))
print(confusion_matrix(y, y_pred))
print("roc auc : ", roc_auc_score(y, y_pred))
print("recall : ", recall_score(y, y_pred))
print("precision : ", precision_score(y, y_pred))
print("f1 score : ",f1 )


accuracy :  0.8445174562010704
[[86561  4443]
 [11505    62]]
roc auc :  0.4782690231388056
recall :  0.005360076078499179
precision :  0.013762486126526082
f1 score :  0.007715281234444998
